# Libraries

In [ ]:
!pip install snap
import snap
import math
import numpy as np
import pandas as pd
import networkx as nx
import csv
from matplotlib import pyplot as plt
import csv
import random
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report,confusion_matrix
from collections import Counter
from tqdm import tqdm

  Created wheel for snap: filename=snap-0.5-py3-none-any.whl size=19398 sha256=73600c24091b3640a4f731af3d13d25b45a64e5f4b2eecb2707b061bbe409585
  Stored in directory: /root/.cache/pip/wheels/3e/5b/9c/8aabe4a690163d236a0010f861eaf746527761a5f4122ab466
Successfully built snap


In [ ]:
# Mouting Google drive for accessing the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Parameters

In [ ]:
# We have used the sample size of 0.1
sample_size = 0.1
test_size = 0.25
scale = True

# Load and prepare data

In [ ]:
df_0 = pd.read_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/080329/0.txt", sep = '\t', header = None)
df_1 = pd.read_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/080329/1.txt", sep = "\t", header = None)
df_2 = pd.read_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/080329/2.txt", sep = "\t", header = None)
df_3 = pd.read_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/080329/3.txt", sep = "\t", header = None)

In [ ]:
cols = ['video_ID', "uploader", "age", "category", "length", "views", "rate", "ratings", "comments", 
        "related_video_1", "related_video_2", "related_video_3", "related_video_4", 
        "related_video_5", "related_video_6", "related_video_7", "related_video_8", 
        "related_video_9", "related_video_10", "related_video_11", "related_video_12",
        "related_video_13", "related_video_14", "related_video_15", "related_video_16",
        "related_video_17", "related_video_18", "related_video_19", "related_video_20"]

In [ ]:
df_0.columns = cols
df_1.columns = cols
df_2.columns = cols
df_3.columns = cols
node_information = pd.concat([df_0, df_1, df_2, df_3])

In [ ]:
node_information.head()

,video_ID,uploader,age,category,length,views,rate,ratings,comments,related_video_1,...,related_video_11,related_video_12,related_video_13,related_video_14,related_video_15,related_video_16,related_video_17,related_video_18,related_video_19,related_video_20
0,kAhMJHat0cI,LisaNova,1137.0,Entertainment,284.0,141189.0,4.30,3090.0,1976.0,RLBx7ccIOkM,...,trcpKpl63Vk,AMUwjdn7Dyg,cDq0B4zEUes,iRkiqb0qAw8,LSHHdD3Kfvk,83mHSIdbKD8,zq_geLMFpf0,X46looQt4kg,-mIe0h3zFVM,D9HeJYGdmRE
1,RTNXfv5zNss,machinima,1137.0,Entertainment,1206.0,123367.0,4.91,2115.0,1822.0,vg_ZkoUteLM,...,2M1Ji9ZijGI,ylzYrQQERNU,ouZOrqhbYlo,Owxw44ONFE8,Hlhfx0wda1Y,qpKbJX9FwZQ,ikJpyjfiBVA,5DbePZddEuU,dNwVEo0YWyI,U8dCpYhN528
2,74N_Yn27ZWc,sxephil,1137.0,Entertainment,275.0,140875.0,4.55,2435.0,1636.0,3oUZSp7PDEw,...,HQoodoBECx0,KC-noDgddJo,oRLpMlVCqZ4,wZeTFrsxKDg,HEYoRm8IlgY,MRVmULZNcf8,sw4b8NejuVM,ZlS_2UNURR8,jKfblDxvV08,7LBEdFWCYlU
3,Oi9VzgDseL8,ukjonasbrothersfan,1137.0,Entertainment,62.0,81526.0,4.84,1089.0,1354.0,ArG3RwRuFjs,...,wlqmzYWx3GM,7UKQCfifJdQ,PG8tzJQ0vok,qBhY3aaR5RM,6q1O07LHG0I,cEG2QFxi8kw,NR8D2udWwmg,7GWo3kyhp-8,hWannl8LuOw,M35thjwHVaw
4,dznIgkR50Os,toniCHRYSA,1137.0,Film & Animation,389.0,108391.0,4.84,702.0,1365.0,u0gLZq-doD0,...,K2pu3JTwrIo,zUtdxCxQjJo,_I49oMFLcwM,ZYsjGiYUHTg,SSeOPuGFxWs,SvkWyelwTBg,KtiI7vcIYc0,qGpFuTggwL8,ngKtowElOx8,DAfV3mbVeec


In [ ]:
node_information = node_information[node_information['age'].notna()]

In [ ]:
node_information = node_information.sample(frac=sample_size)

In [ ]:
node_information.shape

(18648, 29)

# Create graph

In [ ]:
def buildDirectedGraph(df):
    G = nx.DiGraph()
    views = {}
    print("Adding nodes to Graph")
    edgeList = open('youtube.edgelist', 'w') 
    for _, row in df.iterrows():
        nodeID = row[0]
        views[nodeID] = row[5] if row[5] == 0 else math.log(row[5], 10)
        if not G.has_node(nodeID):
            G.add_node(nodeID)

    newToOld = {}
    oldToNew = {}
    newIDIter = 0
    for _, row in df.iterrows():
        edges = row[9:]
        nodeID = row[0]
        if nodeID not in oldToNew:
            oldToNew[nodeID] = newIDIter
            newToOld[newIDIter] = nodeID
            newIDIter += 1
        for edge in edges:
            if not G.has_node(edge):
                continue
            G.add_edge(nodeID, edge)
            if edge not in oldToNew:
                oldToNew[edge] = newIDIter
                newToOld[newIDIter] = edge
                newIDIter += 1
            edgeList.write(str(oldToNew[nodeID]) + ' ' + str(oldToNew[edge]) + '\n')

    print("Graph loaded")
    print(str(nx.number_of_nodes(G)) + " nodes")
    edgeList.close()
    np.save("newToOldMap.npy", newToOld)
    return G, views

In [ ]:
def buildUndirectedGraph(df):
    G = nx.Graph()
    views = {}
    print("Adding nodes to Graph")
    edgeList = open('youtube.edgelist', 'w') 
    for _, row in df.iterrows():
        nodeID = row[0]
        views[nodeID] = row[5] if row[5] == 0 else math.log(row[5], 10)
        if not G.has_node(nodeID):
            G.add_node(nodeID)

    newToOld = {}
    oldToNew = {}
    newIDIter = 0
    for _, row in df.iterrows():
        edges = row[9:]
        nodeID = row[0]
        if nodeID not in oldToNew:
            oldToNew[nodeID] = newIDIter
            newToOld[newIDIter] = nodeID
            newIDIter += 1
        for edge in edges:
            if not G.has_node(edge):
                continue
            G.add_edge(nodeID, edge)
            if edge not in oldToNew:
                oldToNew[edge] = newIDIter
                newToOld[newIDIter] = edge
                newIDIter += 1
            edgeList.write(str(oldToNew[nodeID]) + ' ' + str(oldToNew[edge]) + '\n')

    print("Graph loaded")
    print(str(nx.number_of_nodes(G)) + " nodes")
    edgeList.close()
    np.save("newToOldMap.npy", newToOld)
    return G, views

In [ ]:
G, views = buildDirectedGraph(node_information)

Adding nodes to Graph
Graph loaded
18648 nodes


In [ ]:
G_undirected, views = buildUndirectedGraph(node_information)

Adding nodes to Graph
Graph loaded
18648 nodes


# Create source-target link dataset

In [ ]:
nodelist = [i for i in G.nodes]

In [ ]:
def prepareDatasets(G, nodelist):
  allEdges = list(G.edges())
  edgesSet = set(allEdges)
  numEdges = len(allEdges)
  goalPositiveExamples = 0.5*numEdges
  goalNegativeExamples = 0.5*numEdges
  numPositiveExamples = 0
  numNegativeExamples = 0
  df_linkset = pd.DataFrame()
  source = []
  target = []
  link = []
  nodePairsInDataset = set()
  while numNegativeExamples < goalNegativeExamples:
    node1 = random.choice(nodelist)
    node2 = random.choice(nodelist)
    if node1 == node2:
      continue
    potentialEdge = node1, node2
    if potentialEdge not in edgesSet and potentialEdge not in nodePairsInDataset:
      source.append(node1)
      target.append(node2)
      link.append(0)
      nodePairsInDataset.add(potentialEdge)
      numNegativeExamples += 1
  random.shuffle(allEdges)
  for i in range(int(goalPositiveExamples)):
    edge = allEdges[i]
    source.append(edge[0])
    target.append(edge[1])
    link.append(1)

  df_linkset['source'] = []
  df_linkset['target'] = []
  df_linkset['link'] = []
  df_linkset['source'] = source
  df_linkset['target'] = target
  df_linkset['link'] = link

  return df_linkset

In [ ]:
df_linkset = prepareDatasets(G, nodelist)

In [ ]:
#shuffle the dataframe:
df = df_linkset.sample(frac=1).reset_index(drop=True)

In [ ]:
df.to_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/source_target_links.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10077 entries, 0 to 10076
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  10077 non-null  object
 1   target  10077 non-null  object
 2   link    10077 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 236.3+ KB


In [ ]:
df.head()

,source,target,link
0,rkfzhYMj35k,qN3kC_4xURA,0
1,J85b993voac,UoYLkSk6k04,1
2,r_JC8mFoBPI,PsM7xLkORWg,1
3,U5Ojfo2qkNY,FAOEOgrpRZk,0
4,SjqRGPG2Yas,84Fng-NSWys,0


In [ ]:
df.shape

(10077, 3)

# Node2Vec Embeddings

In [ ]:
!pip install node2vec
from node2vec import Node2Vec as n2v

  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=5fc99c54af09f34daa0d437a330ba061b8a318652537905a0feabfd8fa64948a
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec


In [ ]:
# Time ~ 5 mins for 10% sample
graph_emb = n2v(G_undirected, dimensions = 16)

Computing transition probabilities:   0%|          | 0/18648 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [04:41<00:00, 28.17s/it]


In [ ]:
window_size = 1
minimum_count = 1
batchofwords = 4

model = graph_emb.fit(window=window_size, min_count=minimum_count, batch_words=batchofwords)

In [ ]:
emb_df = (pd.DataFrame([model.wv.get_vector(str(n)) for n in G_undirected.nodes()], 
                       index = G_undirected.nodes()))

In [ ]:
#emb_df['Node'] = G_undirected.nodes()

In [ ]:
emb_df.to_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/node_embedding_10perc_sample.csv", index=True)

In [ ]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
glu82z2gtUE,-0.956772,-1.795235,0.327472,-1.950309,-2.374627,-0.255067,0.357089,2.419735,0.864712,2.360615,-0.258884,-1.660069,-2.121730,-0.490644,0.647794,-0.822499
JiWpxoamaPw,-0.019680,0.001526,0.017977,-0.003966,0.022122,-0.028697,-0.011315,-0.003962,0.025423,0.003222,0.014192,0.029313,0.023789,-0.026975,-0.031161,0.016586
easbmYk6354,-0.016780,0.024632,-0.017792,0.006114,-0.025209,0.021780,0.022251,-0.011884,-0.005356,0.016459,-0.009872,-0.007997,0.030256,-0.007121,0.012483,0.000437
Vt246AnI4N4,0.013505,0.015744,0.015141,-0.006178,0.008078,-0.006827,0.017503,0.000102,-0.010080,0.007186,0.013232,0.002477,-0.030334,-0.005677,0.006790,-0.021627
Mn4Z7lY00rM,-0.003796,-0.027584,0.030226,0.018386,0.000260,-0.025266,-0.029702,-0.017094,-0.011433,0.018266,0.029322,0.025810,-0.003986,0.021151,0.029245,0.008653


In [ ]:
def emb_features(df):
  sums = []
  avgs = []
  prods = []

  for i in tqdm(range(len(df))):
    rep_source = list(np.array(emb_df[emb_df.index == df.iloc[i]['source']])[0])
    rep_target = list(np.array(emb_df[emb_df.index == df.iloc[i]['target']])[0]) 
    sum = [a+b for a,b in zip(rep_source, rep_target)]
    sums.append(np.array(sum))
    avg = [(a+b)/2 for a,b in zip(rep_source, rep_target)]
    avgs.append(np.array(avg))
    prod = [a*b for a,b in zip(rep_source, rep_target)]
    prods.append(np.array(prod))
  emb_sum = pd.DataFrame(sums, columns = ['sum_0', 'sum_1', 'sum_2', 'sum_3', 'sum_4', 'sum_5', 'sum_6', 'sum_7', 'sum_8', 'sum_9', 'sum_10', 'sum_11', 'sum_12', 'sum_13', 'sum_14', 'sum_15'])
  emb_avg = pd.DataFrame(avgs, columns = ['avg_0', 'avg_1', 'avg_2', 'avg_3', 'avg_4', 'avg_5', 'avg_6', 'avg_7', 'avg_8', 'avg_9', 'avg_10', 'avg_11', 'avg_12', 'avg_13', 'avg_14', 'avg_15'])
  emb_prod = pd.DataFrame(prods, columns = ['prod_0', 'prod_1', 'prod_2', 'prod_3', 'prod_4', 'prod_5', 'prod_6', 'prod_7', 'prod_8', 'prod_9', 'prod_10', 'prod_11', 'prod_12', 'prod_13', 'prod_14', 'prod_15'])
  final_df = pd.concat([df, emb_sum, emb_avg, emb_prod], axis=1)
  return final_df

In [ ]:
df = emb_features(df)
df.head()

,source,target,link,sum_0,sum_1,sum_2,sum_3,sum_4,sum_5,sum_6,...,prod_6,prod_7,prod_8,prod_9,prod_10,prod_11,prod_12,prod_13,prod_14,prod_15
0,rkfzhYMj35k,qN3kC_4xURA,0,-1.413013,-1.778044,-0.372599,-1.539065,-2.383949,1.203595,-0.034141,...,-0.000121,0.016491,0.007669,0.047752,0.015984,0.041321,-0.011748,0.025275,0.002080,-0.005934
1,J85b993voac,UoYLkSk6k04,1,-1.626465,-2.162876,-1.854273,-3.671918,-6.044376,0.502586,-2.740177,...,1.289787,3.212803,-1.625392,0.004091,-0.226572,-0.966053,-0.657066,0.724920,-1.298165,2.810447
2,r_JC8mFoBPI,PsM7xLkORWg,1,3.892650,0.486121,-1.137511,-2.722686,-5.124178,3.502509,-4.314734,...,4.624921,0.642589,0.416287,8.001205,4.392770,0.068829,2.667431,0.438939,1.149678,0.007986
3,U5Ojfo2qkNY,FAOEOgrpRZk,0,-1.054151,-1.225188,0.573736,-4.958475,-3.577811,0.855308,-1.804483,...,-1.023890,0.321491,1.472359,0.550672,0.112404,0.261672,-0.786307,-0.126158,-0.929809,2.448215
4,SjqRGPG2Yas,84Fng-NSWys,0,-0.667670,-2.896195,-2.572122,-2.121985,-0.676632,3.341443,-0.198435,...,-1.635641,5.770884,0.554119,2.297662,0.066693,0.828228,-1.184759,1.327445,-0.588942,4.710648


# Creating features

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/source_target_links.csv")

In [ ]:
df.drop("Unnamed: 0", inplace = True, axis = 1)

In [ ]:
df.shape

(10077, 51)

In [ ]:
page_rank = nx.pagerank_scipy(G)
hub_score, authority_score = nx.hits(G)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: networkx.pagerank_scipy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  """Entry point for launching an IPython kernel.


In [ ]:
from networkx.algorithms.link_prediction import preferential_attachment
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def features(df):

  age_diff = []
  jaccard_coef = []
  adamic_adar_coef = []
  pref_attachement_coef = []
  common_neigh = []
  # triad_features = []
  source_indegree = []
  target_indegree = []
  source_outdegree = []
  target_outdegree = []
  pagerank = []
  source_hubs = []
  target_authorities = []

  for i in tqdm(range(len(df))):

    source = df_linkset.iloc[i]['source']
    target = df_linkset.iloc[i]['target']
    idx_source, idx_target = nodelist.index(source), nodelist.index(target)

    age_diff.append(int(list(node_information['age'])[idx_source] - list(node_information['age'])[idx_target]))

    jaccard = nx.jaccard_coefficient(G_undirected, [(source, target)])
    for u, v, p in jaccard:
      jaccard_coef.append(p)

    adamic_adar = nx.adamic_adar_index(G_undirected, [(source, target)])
    for u, v, p in adamic_adar:
      adamic_adar_coef.append(p)

    pref_attachement = nx.preferential_attachment(G_undirected, [(source, target)])
    for u, v, p in pref_attachement:
      pref_attachement_coef.append(p)

    common_neigh.append(len(sorted(nx.common_neighbors(G_undirected, source, target))))

    source_indegree.append(G.in_degree(source))
    source_outdegree.append(G.out_degree(source))
    target_indegree.append(G.in_degree(target))
    target_outdegree.append(G.out_degree(target))

    pagerank.append(page_rank.get(target))
    source_hubs.append(hub_score.get(source))
    target_authorities.append(hub_score.get(target))

  df['age_diff'] = age_diff
  df['jaccard_coef'] = jaccard_coef
  df['adamic_adar_coef'] = adamic_adar_coef
  df['pref_attachment_coef'] = pref_attachement_coef
  df['common_neigh'] = common_neigh
  df['source_indegree'] = source_indegree
  df['source_outdegree'] = source_outdegree
  df['target_indgree'] = target_indegree
  df['target_outdegree'] = target_outdegree
  df['pagerank'] = pagerank
  df['source_hubs'] = source_hubs
  df['target_authorities'] = target_authorities

  return df

In [ ]:
def extend_features(df):

  triad_features_0 = []
  triad_features_1 = []
  triad_features_2 = []
  triad_features_3 = []
  triad_features_4 = []
  triad_features_5 = []
  triad_features_6 = []
  triad_features_7 = []

  for i in tqdm(range(len(df))):   

    source = df.iloc[i]['source']
    target = df.iloc[i]['target']
    idx_source, idx_target = nodelist.index(source), nodelist.index(target)

    triad_feature = [0.0]*8

    for w in sorted(nx.common_neighbors(G_undirected, source, target)):
      if G_undirected.has_edge(source, w) and G_undirected.has_edge(w, target):
        triad_feature[0] += 1
      if G_undirected.has_edge(source, w) and G_undirected.has_edge(target, w):
        triad_feature[1] += 1   
      if G_undirected.has_edge(w, source) and G_undirected.has_edge(w, target):
        triad_feature[2] += 1
      if G_undirected.has_edge(w, source) and G_undirected.has_edge(target, w):
        triad_feature[3] += 1
      
    for i in range(4,8):
      if triad_feature[i-4] != 0:
        triad_feature[i] = triad_feature[i-4]/len(sorted(nx.common_neighbors(G_undirected, source, target)))

    triad_features_0.append(triad_feature[0])
    triad_features_1.append(triad_feature[1])
    triad_features_2.append(triad_feature[2])
    triad_features_3.append(triad_feature[3])
    triad_features_4.append(triad_feature[4])
    triad_features_5.append(triad_feature[5])
    triad_features_6.append(triad_feature[6])
    triad_features_7.append(triad_feature[7])

  df['triad_feature_0'] = triad_features_0
  df['triad_feature_1'] = triad_features_1
  df['triad_feature_2'] = triad_features_2
  df['triad_feature_3'] = triad_features_3
  df['triad_feature_4'] = triad_features_4
  df['triad_feature_5'] = triad_features_5
  df['triad_feature_6'] = triad_features_6
  df['triad_feature_7'] = triad_features_7

  return df

In [ ]:
df = features(df)

100%|██████████| 10077/10077 [01:06<00:00, 151.30it/s]


In [ ]:
df = extend_features(df)

100%|██████████| 10077/10077 [00:15<00:00, 656.76it/s]


In [ ]:
df.head()

,source,target,link,sum_0,sum_1,sum_2,sum_3,sum_4,sum_5,sum_6,...,source_hubs,target_authorities,triad_feature_0,triad_feature_1,triad_feature_2,triad_feature_3,triad_feature_4,triad_feature_5,triad_feature_6,triad_feature_7
0,rkfzhYMj35k,qN3kC_4xURA,0,-1.413013,-1.778044,-0.372599,-1.539065,-2.383949,1.203595,-0.034141,...,-0.0,-0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,J85b993voac,UoYLkSk6k04,1,-1.626465,-2.162876,-1.854273,-3.671918,-6.044376,0.502586,-2.740177,...,-0.0,-0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,r_JC8mFoBPI,PsM7xLkORWg,1,3.892650,0.486121,-1.137511,-2.722686,-5.124178,3.502509,-4.314734,...,-0.0,-0.000000e+00,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
3,U5Ojfo2qkNY,FAOEOgrpRZk,0,-1.054151,-1.225188,0.573736,-4.958475,-3.577811,0.855308,-1.804483,...,-0.0,-0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SjqRGPG2Yas,84Fng-NSWys,0,-0.667670,-2.896195,-2.572122,-2.121985,-0.676632,3.341443,-0.198435,...,-0.0,-2.702190e-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.shape

(10077, 71)

In [ ]:
df.to_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/df_with_features_10perc.csv")

# Train/test split

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/MLNS/Final Project/Data/df_with_features.csv")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(['source', 'target', 'link'], axis=1), df['link'], test_size = 0.1)

# Models

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix, f1_score

In [ ]:
if scale == True:
  X_train, X_test = preprocessing.scale(X_train), preprocessing.scale(X_test)

## Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
lsvc= LinearSVC()
lsvc.fit(X_train, y_train)
lsvc.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


0.8948412698412699

In [ ]:
print(classification_report(y_test, lsvc.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89       524
           1       0.82      1.00      0.90       484

    accuracy                           0.89      1008
   macro avg       0.91      0.90      0.89      1008
weighted avg       0.91      0.89      0.89      1008



In [ ]:
LinearSVC().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [ ]:
# defining parameter range
param_grid = {'C': [0.01, 0.1, 1],
              'penalty': ['l1', 'l2'],
              'dual': [False] ## Documentation says prefer False when n_samples > n_features
              }
 
grid = GridSearchCV(LinearSVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....C=0.01, dual=False, penalty=l1;, score=0.905 total time=   0.4s
[CV 2/5] END ....C=0.01, dual=False, penalty=l1;, score=0.910 total time=   0.3s
[CV 3/5] END ....C=0.01, dual=False, penalty=l1;, score=0.895 total time=   0.3s
[CV 4/5] END ....C=0.01, dual=False, penalty=l1;, score=0.901 total time=   0.3s
[CV 5/5] END ....C=0.01, dual=False, penalty=l1;, score=0.897 total time=   0.3s
[CV 1/5] END ....C=0.01, dual=False, penalty=l2;, score=0.908 total time=   0.1s
[CV 2/5] END ....C=0.01, dual=False, penalty=l2;, score=0.912 total time=   0.1s
[CV 3/5] END ....C=0.01, dual=False, penalty=l2;, score=0.897 total time=   0.4s
[CV 4/5] END ....C=0.01, dual=False, penalty=l2;, score=0.902 total time=   0.4s
[CV 5/5] END ....C=0.01, dual=False, penalty=l2;, score=0.905 total time=   0.2s
[CV 1/5] END .....C=0.1, dual=False, penalty=l1;, score=0.907 total time=   0.9s
[CV 2/5] END .....C=0.1, dual=False, penalty=l1;,

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': [0.01, 0.1, 1], 'dual': [False],
                         'penalty': ['l1', 'l2']},
             verbose=3)

In [ ]:
grid.score(X_test, y_test)

0.8948412698412699

In [ ]:
grid.best_estimator_

LinearSVC(C=1, dual=False, penalty='l1')

## Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF =  RandomForestClassifier(n_estimators= 1000)
RF.fit(X_train, y_train)
RF.score(X_test, y_test)

0.8809523809523809

In [ ]:
# Checking score on training data as well
print(f1_score(list(y_train), list(RF.predict(X_train))))

1.0


In [ ]:
#hyperparameter grid search for randomforest
n_estimators = [200, 400, 600, 800, 1000]
max_depth = [5, 10, 15]
min_samples_split = [2,4,6]
min_samples_leaf = [1, 3, 6]

hyperF = dict(n_estimators = n_estimators, 
              min_samples_split = min_samples_split,
             min_samples_leaf = min_samples_leaf)
clf = GridSearchCV(RandomForestClassifier(), hyperF, cv = 3, verbose = 1,
                      n_jobs = -1, scoring='accuracy')

clf.fit(X_train, y_train)
clf_best = clf.best_estimator_

Fitting 3 folds for each of 45 candidates, totalling 135 fits


KeyboardInterrupt: ignored

In [ ]:
clf_best.score(X_test, y_test)

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

In [ ]:
print(classification_report(y_test, dt.predict(X_test)))

In [ ]:
DecisionTreeClassifier().get_params()

In [ ]:
# defining parameter range
param_grid = {'max_depth': [10,11,12],
              'min_samples_leaf': [1,4,6],
              # 'criterion': ['gini', 'entropy']
              }
 
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
grid.score(X_test, y_test)

In [ ]:
grid.best_estimator_

## Light GBM

In [ ]:
import lightgbm as lgb

In [ ]:
clf = lgb.LGBMClassifier(random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9740382115934494

In [ ]:
lgb.LGBMClassifier().get_params()

In [ ]:
# defining parameter range
param_grid = {'max_depth': [5,10,15],
              'learning_rate': [0.1, 0.01],
              'n_estimators': [100,200,500]
              }
 
grid = GridSearchCV(lgb.LGBMClassifier(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

In [ ]:
grid.best_estimator_

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

abc = AdaBoostClassifier(n_estimators=100)
abc = abc.fit(X_train,y_train)

In [ ]:
y_pred = abc.predict(X_test)
print('F1: %f' %(f1_score(y_pred, y_test)))

In [ ]:
model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

param_grid = {
    'base_estimator__max_depth':[i for i in range(2,11,2)],            
    'base_estimator__min_samples_leaf':[5,10],
    'n_estimators':[10,50,100],
    'learning_rate':[0.01,0.1]
}
abc_hp = GridSearchCV(estimator=model,param_grid=param_grid, cv=5, n_jobs=-1,verbose=2)
abc_hp.fit(X_train,y_train)

In [ ]:
print(abc_hp.best_params_)

In [ ]:
abc = AdaBoostClassifier(**abc_hp.best_params_)
abc = abc.fit(X_train, y_train)

In [ ]:
y_pred = abc.predict(X_test)
print('F1: %f' %(f1_score(y_pred, y_test)))

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc = gbc.fit(X_train,y_train)

In [ ]:
y_pred = gbc.predict(X_test)
print('F1: %f' %(f1_score(y_pred, y_test)))

In [ ]:
param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.1],
    "min_samples_split": np.linspace(0.1, 0.5, 3),
    "min_samples_leaf": np.linspace(0.1, 0.5, 3),
    "max_depth":[i for i in range(2,11,2)],
    "n_estimators":[10, 100]
}

gbc_hp = GridSearchCV(estimator = gbc, param_grid = param_grid, cv = 5)
gbc_hp.fit(X_train, y_train)


In [ ]:
gbc_hp.best_params_

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(**gbc_hp.best_params_)
gbc = gbc.fit(X_train, y_train)

In [ ]:
y_pred = gbc.predict(X_test)
print('F1: %f' %(f1_score(y_pred, y_test)))

F1: 0.972644


## XGBoost


In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators=1000)
xgb.fit(X_train, y_train)

XGBClassifier(n_estimators=1000)

In [ ]:
xgb.score(X_test, y_test)

In [ ]:
# checking score on training data as well

print(f1_score(list(y_train), list(xgb.predict(X_train))))

## Multi layer perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
parameters = {'solver':['sgd','adam'],
              'alpha':[0.0001, 0.0005, 0.005],
              'activation':['relu', 'tanh'],
              'hidden_layer_sizes':[(60,20), (30, 10)],
              'tol':[5e-5],
              'max_iter':[150],
              'verbose':[1]
              }
clf = GridSearchCV(mlp, parameters)

In [ ]:
clf.fit(X_train, y_train)

Iteration 1, loss = 0.59724413
Iteration 2, loss = 0.46833360
Iteration 3, loss = 0.40067336
Iteration 4, loss = 0.35856110
Iteration 5, loss = 0.32900461
Iteration 6, loss = 0.30741508
Iteration 7, loss = 0.29127450
Iteration 8, loss = 0.27911847
Iteration 9, loss = 0.26982294
Iteration 10, loss = 0.26253208
Iteration 11, loss = 0.25678660
Iteration 12, loss = 0.25215458
Iteration 13, loss = 0.24835803
Iteration 14, loss = 0.24517558
Iteration 15, loss = 0.24247343
Iteration 16, loss = 0.24011018
Iteration 17, loss = 0.23804221
Iteration 18, loss = 0.23620073
Iteration 19, loss = 0.23459178
Iteration 20, loss = 0.23311388
Iteration 21, loss = 0.23185493
Iteration 22, loss = 0.23054421
Iteration 23, loss = 0.22946742
Iteration 24, loss = 0.22837515
Iteration 25, loss = 0.22743515
Iteration 26, loss = 0.22652166
Iteration 27, loss = 0.22566825
Iteration 28, loss = 0.22487724
Iteration 29, loss = 0.22412827
Iteration 30, loss = 0.22343590
Iteration 31, loss = 0.22273802
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 3, loss = 0.41883618
Iteration 4, loss = 0.36057895
Iteration 5, loss = 0.32175126
Iteration 6, loss = 0.29588151
Iteration 7, loss = 0.27824629
Iteration 8, loss = 0.26583555
Iteration 9, loss = 0.25672997
Iteration 10, loss = 0.24986242
Iteration 11, loss = 0.24459878
Iteration 12, loss = 0.24042685
Iteration 13, loss = 0.23699884
Iteration 14, loss = 0.23414142
Iteration 15, loss = 0.23175799
Iteration 16, loss = 0.22967706
Iteration 17, loss = 0.22790030
Iteration 18, loss = 0.22634162
Iteration 19, loss = 0.22489063
Iteration 20, loss = 0.22374084
Iteration 21, loss = 0.22257855
Iteration 22, loss = 0.22161721
Iteration 23, loss = 0.22068239
Iteration 24, loss = 0.21978983
Iteration 25, loss = 0.21893176
Iteration 26, loss = 0.21817798
Iteration 27, loss = 0.21748399
Iteration 28, loss = 0.21679472
Iteration 29, loss = 0.21614825
Iteration 30, loss = 0.21555143
Iteration 31, loss = 0.21496679
Iteration 32, loss = 0.21440408
Iteration 33, loss = 0.21388941
Iteration 34, l

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 3, loss = 0.40848397
Iteration 4, loss = 0.35582996
Iteration 5, loss = 0.32068836
Iteration 6, loss = 0.29684322
Iteration 7, loss = 0.28017073
Iteration 8, loss = 0.26812105
Iteration 9, loss = 0.25913964
Iteration 10, loss = 0.25221608
Iteration 11, loss = 0.24682617
Iteration 12, loss = 0.24241017
Iteration 13, loss = 0.23888294
Iteration 14, loss = 0.23579037
Iteration 15, loss = 0.23322659
Iteration 16, loss = 0.23100957
Iteration 17, loss = 0.22909358
Iteration 18, loss = 0.22737844
Iteration 19, loss = 0.22585868
Iteration 20, loss = 0.22448023
Iteration 21, loss = 0.22323425
Iteration 22, loss = 0.22210550
Iteration 23, loss = 0.22104351
Iteration 24, loss = 0.22009806
Iteration 25, loss = 0.21921208
Iteration 26, loss = 0.21837998
Iteration 27, loss = 0.21761270
Iteration 28, loss = 0.21684113
Iteration 29, loss = 0.21611061
Iteration 30, loss = 0.21542263
Iteration 31, loss = 0.21477032
Iteration 32, loss = 0.21422551
Iteration 33, loss = 0.21352108
Iteration 34, l

...

Iteration 1, loss = 0.43349583
Iteration 2, loss = 0.26223598
Iteration 3, loss = 0.23646694
Iteration 4, loss = 0.22633643
Iteration 5, loss = 0.22005060
Iteration 6, loss = 0.21622802
Iteration 7, loss = 0.21283408
Iteration 8, loss = 0.21011540
Iteration 9, loss = 0.20734322
Iteration 10, loss = 0.20483669
Iteration 11, loss = 0.20309712
Iteration 12, loss = 0.20020478
Iteration 13, loss = 0.19780761
Iteration 14, loss = 0.19563366
Iteration 15, loss = 0.19383526
Iteration 16, loss = 0.19187975
Iteration 17, loss = 0.19009104
Iteration 18, loss = 0.18765591
Iteration 19, loss = 0.18674552
Iteration 20, loss = 0.18443665
Iteration 21, loss = 0.18253897
Iteration 22, loss = 0.18110423
Iteration 23, loss = 0.18015707
Iteration 24, loss = 0.17742748
Iteration 25, loss = 0.17544747
Iteration 26, loss = 0.17335665
Iteration 27, loss = 0.17229266
Iteration 28, loss = 0.16998064
Iteration 29, loss = 0.16855366
Iteration 30, loss = 0.16726430
Iteration 31, loss = 0.16565905
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Iteration 1, loss = 0.57220645
Iteration 2, loss = 0.34911748
Iteration 3, loss = 0.29282000
Iteration 4, loss = 0.26814296
Iteration 5, loss = 0.25483032
Iteration 6, loss = 0.24675926
Iteration 7, loss = 0.24143827
Iteration 8, loss = 0.23764481
Iteration 9, loss = 0.23466200
Iteration 10, loss = 0.23233978
Iteration 11, loss = 0.23041697
Iteration 12, loss = 0.22876809
Iteration 13, loss = 0.22735309
Iteration 14, loss = 0.22610099
Iteration 15, loss = 0.22498351
Iteration 16, loss = 0.22400455
Iteration 17, loss = 0.22313001
Iteration 18, loss = 0.22222638
Iteration 19, loss = 0.22147287
Iteration 20, loss = 0.22077793
Iteration 21, loss = 0.22011710
Iteration 22, loss = 0.21950517
Iteration 23, loss = 0.21889368
Iteration 24, loss = 0.21837656
Iteration 25, loss = 0.21779706
Iteration 26, loss = 0.21731642
Iteration 27, loss = 0.21685102
Iteration 28, loss = 0.21640196
Iteration 29, loss = 0.21595237
Iteration 30, loss = 0.21550420
Iteration 31, loss = 0.21511412
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.0005, 0.005],
                         'hidden_layer_sizes': [(60, 20), (30, 10)],
                         'max_iter': [150], 'solver': ['sgd', 'adam'],
                         'tol': [5e-05], 'verbose': [1]})

In [ ]:
clf.best_params_

{'activation': 'tanh',
 'alpha': 0.0005,
 'hidden_layer_sizes': (60, 20),
 'max_iter': 150,
 'solver': 'sgd',
 'tol': 5e-05,
 'verbose': 1}

In [ ]:
print(f1_score(list(y_test), list(clf.predict(X_test))))

0.9000951474785918
